In [6]:
import os

import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

In [1]:
# pip install --upgrade google_auth_oauthlib

In [1]:
# pip install --upgrade Youtube

In [5]:
CLIENT_SECRETS_FILE = 'client_secret.json'

SCOPES = ['https://www.googleapis.com/auth/youtube']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

ACTIONS = ('get', 'list', 'set')

# Authorize the request and store authorization credentials.
def get_authenticated_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    credentials = flow.client_config()
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [6]:
get_authenticated_service()

TypeError: 'dict' object is not callable

In [ ]:
# Call the API's playlists.update method to update an existing playlist's default language,
# localized title and description in a specific language.
def set_playlist_localization(youtube, args):

  results = youtube.playlists().list(
    part='snippet,localizations',
    id=args.playlist_id
  ).execute()

  playlist = results['items'][0]

  # If the language argument is set, set the localized title and description
  # for that language. The "title" and "description" arguments have default
  # values to make the script simpler to run as a demo. In an actual app, you
  # would likely want to set those arguments also.
  if args.language and args.language != '':
    if 'localizations' not in playlist:
      playlist['localizations'] = {}

    playlist['localizations'][args.language] = {
      'title': args.title,
      'description': args.description
    }

  # If the default language is set AND there is localized metadata for that
  # language, set the video's title and description to match the localized
  # title and description for the newly set default language.
  if args.default_language:
    playlist['snippet']['defaultLanguage'] = args.default_language
    if args.default_language in playlist['localizations']:
      playlist['snippet']['title'] = (
          playlist['localizations'][args.default_language]['title'])
      playlist['snippet']['description'] = (
          playlist['localizations'][args.default_language]['description'])

  print(playlist)


In [7]:
CLIENT_SECRETS_FILE = 'client_secret.json'

yt = Youtube

ModuleNotFoundError: No module named 'youtube'

In [8]:
import gdata.youtube
import gdata.youtube.service

AttributeError: 'function' object has no attribute 'func_name'

In [7]:
yt_service = gdata.youtube.service.YouTubeService()
yt_service.ssl = True

# a typical playlist URI
playlist_uri = "http://gdata.youtube.com/feeds/api/playlists/PLCD939C4D974A5815"

playlist_video_feed = yt_service.GetYouTubePlaylistVideoFeed(playlist_uri)

# iterate through the feed as you would with any other
for playlist_video_entry in playlist_video_feed.entry:
    print(playlist_video_entry.title.text)

NameError: name 'gdata' is not defined

In [11]:
channel_name="josephfox9526"
playlist_name="PLlfOdKRE3hkDUPTg2u2x4E18SQxUpRyxK"

from currentsecrets import youtube_token


api_key = youtube_token

from apiclient.discovery import build
import pandas as pd

youtube = build('youtube', 'v3', developerKey=api_key)



def get_channel_videos(channel_id):

# get Uploads playlist id
    res = youtube.channels().list(id=channel_id, 
                              part='contentDetails').execute()
    playlist_id = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    videos = []
    next_page_token = None

    while 1:
        res = youtube.playlistItems().list(playlistId=playlist_name, 
                                       part='snippet', 
                                       maxResults=50,
                                       pageToken=next_page_token).execute()
        videos += res['items']
        next_page_token = res.get('nextPageToken')

        if next_page_token is None:
            break

    return videos

videos = get_channel_videos(channel_name)

KeyError: 'items'

In [ ]:
def get_videos_stats(video_ids):
stats = []
for i in range(0, len(video_ids), 50):
    res = youtube.videos().list(id=','.join(video_ids[i:i+50]),
                               part='statistics').execute()
    stats += res['items']

return stats

video_ids = list(map(lambda x:x['snippet']['resourceId']['videoId'], videos))

stats = get_videos_stats(video_ids)


d = []
if len(stats)!=len(videos):
    i=1
    j=0
else:
    i=0
    j=0
len_video = len(videos)
len_stats = len(stats)
for video in videos:
    if i >= len_video:
        break
    Url_video='https://www.youtube.com/watch?v='+videos[i]['snippet']['resourceId']['videoId']+'&list='+playlist_name
    d.append((videos[i]['snippet']['title'], 
              videos[i]['snippet']['resourceId']['videoId'], 
              Url_video, 
              stats[j]['statistics']['viewCount'],
              stats[j]['statistics']['likeCount'],
              stats[j]['statistics']['dislikeCount']
              ))

    i+=1
    j+=1


df=pd.DataFrame(d, columns=('Titre_video', 'ID_video', 'Url_video','vues','like','dislike'))
df['vues'] = df['vues'].astype(int)
df['like'] = df['like'].astype(int)
df['dislike'] = df['dislike'].astype(int)
df.index+=1

df.to_excel("youtube-playlist.xlsx")